In [ ]:
import pandas as pd
import datetime, time
import pickle, os, vitaldb
import numpy as np


df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)

print(len(df_match))
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''


error_list=[]
srate= 250
for i in range(len(df_match)):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    vital_path = '../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename
    
    if os.path.exists(vital_path):
    #if not os.path.exists('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename):
        print(i, end=' ')

        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_full_vital/'+df_match.loc[i,'path']
        
        
        # vitaldb 에러에 해당되는 케이스
        df_vital = pickle.load(open(vital_path,'rb'))
        
        if np.sum(~np.isnan(df_vital['ECG'].tolist())) != 0:
            if np.nanmean(df_vital['ECG'].tolist()) > -4:
                print('already done')
                continue
        
        
        error_list.append(filename)
        print(df_match.loc[i,'path'], end='... ')
        
        vfile_ecg = vitaldb.vital_recs_time(rec_path,['SNUADCW/ECG_II'],interval=0.004).flatten()

        
        if len(vfile_ecg[0])==0 :
            print('index ',df_match.loc[i,'index'],' OPID ',df_match.loc[i,'opid'],'empty vital file')
            
        else:
            Abs_time = [vfile_ecg[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=i/srate) for i in range(len(vfile_ecg[0]))]
            #print(vfile[1])
            dics = {'Abs_time':Abs_time,'ECG':vfile_ecg[0]}
            df_vital = pd.DataFrame(data=dics)
            end_idx_list = df_vital[(df_vital['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_vital['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

            
            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")

            else:
                end_idx=end_idx_list[0]+1
                start_idx= end_idx - 5 * 60 * srate
                start_idx_n = max(0,start_idx)

                ext = df_vital[start_idx_n:end_idx]
                extr=ext.dropna(subset=['ECG'])
                extr=extr.drop_duplicates(subset=['ECG'])

                if len(extr)>1:
                    if start_idx <0 :
                        fit = (-1)*end_idx 
                        extraction = pd.DataFrame(index=range(5*60*srate),columns=['ECG'])
                        

                        col_ecg = np.full(5*60*srate, np.nan)
                        col_ecg[fit:] = ext['ECG'][fit:].tolist()                    
                        extraction['ECG'] = col_ecg                        
                        

                    else:
                        extraction = ext
                        
                    with open('../../../cranberry2/Preprocessing/vital_data/ECG_250Hz_pacu_5min/'+filename,'wb') as f:
                        pickle.dump(extraction, f)
                    print('...done')
                else:
                    print('...결측치/직선 데이터')
                
        #except:
            #print('error: ',i,df_match.loc[i,'opid'],df_match.loc[i,'path'])

In [ ]:
import itertools as it
import os
import pandas as pd
import numpy as np
import random
import vitaldb
from pyvital.pyvital import arr
import pickle
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
import time, datetime
import neurokit2 as nk
from tqdm import tqdm

SRATE = 300
LEN_INPUT = 30
STRIDE = 10
LEN_PER_PRE = 30
LEN_PER_POST = 30


# 피크 사이 wave를 모두 같은 length로 만들기 위한 함수
def linear_connection(list, idx):
    int_idx = int(idx)
    return list[int_idx] + (list[int_idx+1] - list[int_idx]) * (idx - int_idx)

def lowess(y, f=0.2):
    x = np.arange(0, len(y))
    return sm.nonparametric.lowess(y, x, frac=f, it=0)[:, 1].T
## 60초짜리 inputp에 대해 f 그려보기


def ECG_filter(seg, method, srate):
    if method == '3rd-bandpass':
        import scipy.signal
        sos = scipy.signal.butter(3, [1,47], 'bandpass', output='sos', fs=srate)
        return scipy.signal.sosfilt(sos, seg)
    # lowess랑 수학적으로 동일한데 더 빠름 - 대신 계수 체크 필요
    elif method == 'savgol':
        import scipy.signal
        return seg - scipy.signal.savgol_filter(seg, 1151, 3)
    
    elif method == 'lowess':
        import statsmodels.api as sm
        x = np.arange(0, len(seg))
        return seg - sm.nonparametric.lowess(seg, x, frac=0.2, it=0)[:, 1].T

    else:
        print('not a valid filter')
        return seg

    
def PPG_filter(seg, method, srate):
    if method == 'bandpass':
        import scipy.signal
        sos = scipy.signal.butter(5, [1,40], 'bandpass', output='sos', fs=srate)
        return scipy.signal.sosfilt(sos, seg)
    
    elif method == 'lowess':
        import statsmodels.api as sm
        x = np.arange(0, len(seg))
        return seg - sm.nonparametric.lowess(seg, x, frac=0.2, it=0)[:, 1].T
    
    else:
        print('not a valid filter')
        return seg
    
    
def quality_assessment(seg_ppg, seg_ecg, corr_thres):
    # peak detection
    try:
        min_peak, ppg_peak = arr.detect_peaks(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), SRATE)
        signals, info = nk.ecg_peaks(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), sampling_rate = SRATE)
        ecg_peak = info["ECG_R_Peaks"]

    except Exception as e:
        #print('error of', e)
        return False
        
    if len(ppg_peak)==0 or len(ecg_peak)==0:
        #print('no peak')
        return False


    # segment 내의 ppg, ecg peak idx
    idx_ppg_peak = ppg_peak
    idx_ecg_peak = ecg_peak


    # peak가 HR 30~150 -> 20s - min 10 peaks(HR30)
    # peak 개수가 기준 미달이면 noise 계산 자세히 할 필요없이 False - 이 경우의 noise_info는 -2로 처리
    if len(idx_ppg_peak)<5/10*LEN_INPUT or len(idx_ecg_peak)<5/10*LEN_INPUT:
        #print(' too less peaks', end='')
        return False
    

    # peak와 peak 사이 interval에 대한 noise 여부 -> 따라서 길이는 peak - 1 (noise면 True)
    bool_noise_ppg = [False for k in range(len(idx_ppg_peak)-1)]
    bool_noise_ecg = [False for k in range(len(idx_ecg_peak)-1)]


    #  2.1 peak 간격 이상한 noise (HR 30~150 -> HBI 0.4s ~ 2s로 SRATE 곱해주면 40~200)
    for k in range(len(bool_noise_ppg)):
        if not 0.4*SRATE < idx_ppg_peak[k+1] - idx_ppg_peak[k] < 2*SRATE:
            bool_noise_ppg[k] = True
    for k in range(len(bool_noise_ecg)):
        if not 0.4*SRATE < idx_ecg_peak[k+1] - idx_ecg_peak[k] < 2*SRATE:
            bool_noise_ecg[k] = True


    # 2.2 모양 이상한 noise
    # wave interval into same length(2s(200))
    len_wave = 2*SRATE
    norm_seg_ppg, norm_seg_ecg = [], []

    for k in range(len(bool_noise_ppg)):
        len_interval_ppg = idx_ppg_peak[k+1] - idx_ppg_peak[k]
        # peak 사이 wave를 모두 같은 길이로 변환
        norm_seg_ppg.append([linear_connection(seg_ppg[idx_ppg_peak[k]:idx_ppg_peak[k+1]+1], n/len_wave*len_interval_ppg) for n in range(len_wave)])

    for k in range(len(bool_noise_ecg)):
        len_interval_ecg = idx_ecg_peak[k+1] - idx_ecg_peak[k]
        # peak 사이 wave를 모두 같은 길이로 변환
        norm_seg_ecg.append([linear_connection(seg_ecg[idx_ecg_peak[k]:idx_ecg_peak[k+1]+1], n/len_wave*len_interval_ecg) for n in range(len_wave)])

    # wave interval 사이 correlation 계산 - PPG
    corr_ppg = []
    mean_wave_ppg = np.nanmean(norm_seg_ppg, axis = 0)
    mean_wave_ppg = pd.DataFrame(mean_wave_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
    norm_seg_ppg = pd.DataFrame(norm_seg_ppg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
    for k in range(len(bool_noise_ppg)):
        corr = np.corrcoef(norm_seg_ppg[k], mean_wave_ppg)[0,1]
        corr_ppg.append(corr)
    noise_ppg_perc = np.mean(bool_noise_ppg | (np.array(corr_ppg) < corr_thres))


    # wave interval 사이 correlation 계산 - ECG   
    corr_ecg = []
    mean_wave_ecg = np.nanmean(norm_seg_ecg, axis = 0)
    mean_wave_ecg = pd.DataFrame(mean_wave_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
    norm_seg_ecg = pd.DataFrame(norm_seg_ecg).fillna(method='ffill', axis=1).fillna(method='bfill', axis=1).values
    for k in range(len(bool_noise_ecg)):
        corr = np.corrcoef(norm_seg_ecg[k], mean_wave_ecg)[0,1]
        corr_ecg.append(corr)
    noise_ecg_perc = np.mean(bool_noise_ecg | (np.array(corr_ecg) < corr_thres))                          

    
    # segment의 noise 비율 정보를 return
    return [noise_ppg_perc, noise_ecg_perc]


def preprocess(file_path, SRATE, ECG_FILT, PPG_FILT, CORR_THRES):
    ### Preprocess of input : Denosing -> Quality Assessment
    ### stores the result of preprocess in dataframe
    # path for cache
    if not os.path.exists('./cache'):
        os.mkdir('./cache')     
    if not os.path.exists('./cache/preprocess'):
        os.mkdir('./cache/preprocess')

    hyper_path = f'SRATE{SRATE}_LEN{LEN_INPUT}-PRE{LEN_PER_PRE}-POST{LEN_PER_POST}_STR{STRIDE}_PPG-{PPG_FILT}_ECG-{ECG_FILT}'
    input_path = f"../DL_model/dataset/PD_{hyper_path}/"
    if not os.path.exists('../DL_model/dataset'):
        os.mkdir('../DL_model/dataset')
    if not os.path.exists(input_path[:-1]):
        os.mkdir(input_path[:-1])  
        

    # dataframe to save preprocessing info
    n_aug = int((LEN_PER_PRE-LEN_INPUT)/STRIDE) + 1
    column_list = ['caseid'] + [f'{i+1}' for i in np.arange(n_aug)] + ['age', 'gender']
    df_preprocess = pd.DataFrame(columns = column_list)

    # df_preprocess에 demographs(age, gender) 추가
    df_demograph = pd.read_csv("https://api.vitaldb.net/cases")

    
    
    # set variables
    caseids = os.listdir(file_path)
    f_num = 0
    initial, interval = f_num, len(caseids)
    
    # variables
    non_lis, low_mbp = [], []
    x, x_rftn, y, c = [], [], [], []
    age, gender, y_mbp = [], [], []
    nan_hr, nan_mbp, nan_rftn = 0, 0, 0

    start = time.time()
    pbar = tqdm(caseids[initial:initial+interval])
    for caseid in pbar:
        caseid = caseid[:-4]   # '123.npz' -> '123'
        f_num += 1

        # vital data 불러오기
        try:
            vals = np.load(f'{file_path}/{caseid}.npz')
            
        except Exception as e:
            continue

        
        # baseline 혈압이 낮은 경우(<65mmHg)는 제외: ephedrine, phenylephrine 투여 가능성
        nmbp = pd.DataFrame(vals['nMBP']).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
        if len(nmbp) == 0:
            nan_mbp = nan_mbp + 1
            continue
        if nmbp[-1] < 65:
            low_mbp.append(caseid)
            continue
            

        ### before intubation event (-120 ~ -60sec) preprocessing  
        #n_aug = int((LEN_PER_PRE-LEN_INPUT)/STRIDE) + 1
        segs_ppg = vals['nPPG'][-LEN_PER_PRE*SRATE:]
        segs_ecg = vals['nECG'][-LEN_PER_PRE*SRATE:]
        for i in range(n_aug):
            # vital data
            seg_ppg = segs_ppg[i*STRIDE*SRATE:i*STRIDE*SRATE+LEN_INPUT*SRATE]
            seg_ecg = segs_ecg[i*STRIDE*SRATE:i*STRIDE*SRATE+LEN_INPUT*SRATE]
            
            if len(seg_ppg) == 0 or len(seg_ecg) == 0:
                #print('not valid ecg, ppg', end='...')
                continue

            seg_ppg = [np.nan if ele <=-200 else ele for ele in seg_ppg]
            seg_ecg = [np.nan if ele <=-4 else ele for ele in seg_ecg]

            #dataframe에 새로운 행 만들기
            df_preprocess.loc[f_num-1,'caseid'] = caseid
            
            row_demo = df_demograph[df_demograph['caseid']==int(caseid)]
            df_preprocess.loc[f_num-1, 'age'] = row_demo['age'].values[0]
            df_preprocess.loc[f_num-1, 'gender'] = row_demo['sex'].values[0]

            
            ## 1. 결측치 제거 ##
            nan_ppg_perc = np.mean(np.isnan(seg_ppg))
            nan_ecg_perc = np.mean(np.isnan(seg_ecg))

            # segment의 결측치 비율 정보
            nan_info = [nan_ppg_perc, nan_ecg_perc]

            # 결측치가 많은 경우, noise 확인할 것도 없이 False -  이 경우의 noise_info는 -1로 처리
            if nan_ppg_perc > 0.05 or nan_ecg_perc > 0.05:
                #print(' too much missing data', end='...')
                df_preprocess.loc[f_num-1,f'{i+1}'] = (False, nan_info, [-1, -1])
                continue


            ## 2. Denosing ##
            # ECG : 1-40Hz bandpass filter / PPG : loess filter
            seg_ecg = ECG_filter(pd.DataFrame(seg_ecg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), method=ECG_FILT, srate=SRATE)
            seg_ppg = PPG_filter(pd.DataFrame(seg_ppg).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten(), method=PPG_FILT, srate=SRATE)


            ## 3. Quality Assessment ## 
            # segment의 noise 비율 정보
            noise_info = quality_assessment(seg_ppg, seg_ecg, CORR_THRES) 
            
            # Fail the preprocess
            if not noise_info:
                df_preprocess.loc[f_num-1,f'{i+1}'] = (False, nan_info, [-2, -2])
                continue


            # segment를 input으로 써도 되는지
            bool_pass = [True if noise_info[0] < 0.1 and noise_info[1] < 0.1 else False]


            # 이 segment의 정보를 dataframe에 저장 - (전처리 성공여부, 전처리 nan 비율, 전처리 noise 비율, 통증 점수)
            df_preprocess.loc[f_num-1,f'{i+1}'] = (bool_pass[0], nan_info, noise_info)
            #print(f'preprocessing done...{bool_pass[0]}', end='')
            
            
            if bool_pass[0]:
                rftn = vals['nRFTN'][-LEN_PER_POST*SRATE:][i*STRIDE*SRATE:i*STRIDE*SRATE+LEN_INPUT*SRATE]
                rftn = pd.DataFrame(rftn).fillna(method='ffill', axis=0).fillna(method='bfill', axis=0).values.flatten()
                rftn = rftn / 8
                if np.mean(np.isnan(rftn)) != 0:
                    nan_rftn = nan_rftn + 1
                    continue

                # z-score normalization
                ppg_input = stats.zscore(seg_ppg)
                ecg_input = stats.zscore(seg_ecg)

                
                hr = vals['HR'][:LEN_PER_POST*SRATE][i*STRIDE*SRATE:i*STRIDE*SRATE+LEN_INPUT*SRATE]
                nhr = vals['nHR'][-LEN_PER_POST*SRATE:][i*STRIDE*SRATE:i*STRIDE*SRATE+LEN_INPUT*SRATE]
                if np.nanmean(hr) == 0 or np.nanmean(nhr) == 0:
                    #print(f'abnormal HR value of caseid {caseid}') 
                    nan_hr = nan_hr + 1
                    continue
                del_hr = np.nanmean(hr) / np.nanmean(nhr) - 1
                
                if np.isnan(del_hr):
                    #print(f'  del_hr of caseid {caseid} value is nan')
                    nan_hr = nan_hr + 1
                    continue

                # MBP
                del_mbp = np.nanmean(vals['MBP'][:LEN_PER_POST*SRATE][i*STRIDE*SRATE:i*STRIDE*SRATE+LEN_INPUT*SRATE]) / np.nanmean(vals['nMBP'][-LEN_PER_POST*SRATE:][i*STRIDE*SRATE:i*STRIDE*SRATE+LEN_INPUT*SRATE]) - 1
                if np.isnan(del_mbp):
                    nan_mbp = nan_mbp + 1
                y_mbp.append(del_mbp)
                
                age.append(int(row_demo['age'].values[0]))
                if row_demo['sex'].values[0]=='F':
                    gender.append(1)
                else:
                    gender.append(0)
                x.append([ppg_input, ecg_input])
                x_rftn.append(rftn)
                y.append(del_hr)
                c.append(caseid)

    print(f'\ndumping cache of df_preprocess {f_num}/{len(caseids)}', end='...')


    df_preprocess.reset_index(drop=True, inplace=True)    
    pickle.dump(df_preprocess, open(f'cache/preprocess/df_preprocess_{hyper_path}', 'wb'))
    print('dumping success')
    
    # dataset
    x = np.array(x, np.float32)
    x_rftn = np.array(x_rftn, np.float32)
    y = np.array(y, np.float32)
    y_mbp = np.array(y_mbp)
    c = np.array(c)
    age = np.array(age, int)
    gender = np.array(gender, int)

    # 알맞게 input 변환
    x = np.transpose(x, [0,2,1])

    print('after concatenate + transpose')
    print('x shape:', x.shape)
    print('rftn shape:', x_rftn.shape)
    print('y shape:', y.shape)
    print('caseid num: ', len(c))
    print(f'input path: {input_path}\n')
    print(datetime.datetime.now())


    # train, test set
    caseids = list(np.unique(c))
    if len(caseids) != len(c):
        print('overlapped caseid exists')
    random.shuffle(caseids)

    ntest = max(1, int(len(caseids) * 0.1))
    ntrain = len(caseids) - ntest

    caseid_train = caseids[:ntrain]
    caseid_test = caseids[ntrain:]

    print('전체 caseid 수: {}'.format(len(caseids)))
    print('train caseid 수: {}, test caseid 수: {}'.format(len(caseid_train), len(caseid_test)))

    pickle.dump(caseid_train, open(f'../DL_model/pd/caseid_train_{hyper_path}','wb'))
    pickle.dump(caseid_test, open(f'../DL_model/pd/caseid_test_{hyper_path}','wb'))

    train_mask = np.isin(c, caseid_train)
    test_mask = np.isin(c, caseid_test)

    x_train = x[train_mask]
    x_test = x[test_mask]
    y_train = y[train_mask]
    y_test = y[test_mask]
    mbp_train = y_mbp[train_mask]
    mbp_test = y_mbp[test_mask]
    rftn_train = x_rftn[train_mask]
    rftn_test = x_rftn[test_mask]
    c_train = c[train_mask]
    c_test = c[test_mask]

    age_train = age[train_mask]
    age_test = age[test_mask]
    gender_train = gender[train_mask]
    gender_test = gender[test_mask]


    # 저장하기
    print('saving...', end='', flush=True)
    np.savez_compressed(input_path+'x_train.npz', x_train)
    np.savez_compressed(input_path+'x_test.npz', x_test)
    np.savez_compressed(input_path+'rftn_train.npz', rftn_train)
    np.savez_compressed(input_path+'rftn_test.npz', rftn_test)
    np.savez_compressed(input_path+'y_train.npz', y_train)
    np.savez_compressed(input_path+'y_test.npz', y_test)
    np.savez_compressed(input_path+'mbp_train.npz', mbp_train)
    np.savez_compressed(input_path+'mbp_test.npz', mbp_test)
    np.savez_compressed(input_path+'c_train.npz', c_train)
    np.savez_compressed(input_path+'c_test.npz', c_test)

    np.savez_compressed(input_path+'age_train.npz', age_train)
    np.savez_compressed(input_path+'age_test.npz', age_test)
    np.savez_compressed(input_path+'gender_train.npz', gender_train)
    np.savez_compressed(input_path+'gender_test.npz', gender_test)

    print('done', flush=True)
    print('\nsize of training set:', len(x_train))
    print('size of test set:', len(x_test)) 
    print(f'num of hr value nan : {nan_hr}')
    print(f'num of mbp value nan : {nan_mbp}')
    print(f'num of rftn value nan : {nan_rftn}')
    print(f'num of low baseline bp : {len(low_mbp)}')

    print(datetime.datetime.now())
    print(f'total elapsed time for preprocess: {time.time() - start:.2f}s\n')


    # 전처리 통과 비율 출력
    nl_pass, l_pass = [], []
    for idx, row in df_preprocess.iterrows():
        for i in range(n_aug):
            nl_pass.append(row[f'{i+1}'][0])
        #l_pass.append(row['2'][0])

    print(f'전처리 성공 비율 : intubation 직전 {np.mean(nl_pass)*100:.2f}%') #, intubation 직후 {np.mean(l_pass)*100:.2f}%')
    print(datetime.datetime.now())

In [ ]:
SRATE = 300
ECG_FILT = '3rd-bandpass'
PPG_FILT = 'lowess'
CORR_THRES = 0.9 # cutoff
LEN_INPUT = 30 # length of the input
STRIDE = 1
LEN_PER_PRE = 30 # before intubation
LEN_PER_POST = 30

file_path = f'vital_to_np_pd_{SRATE}Hz'

preprocess(file_path, SRATE, ECG_FILT, PPG_FILT, CORR_THRES)